In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append(os.path.realpath('..'))

In [ ]:
import cache_magic
import numpy as np
import scipy as sp
import pandas as pd
import plotly.express as ex
import plotly.graph_objects as go
import concurrent.futures
from copy import deepcopy

from tracking_v2.target import ConstantVelocityTarget, SingleTurnTarget
from tracking_v2.kalman import LinearKalmanFilter, CoordinatedTurn
from tracking_v2.motion import ConstantVelocityModel, ConstantAccelerationModel, SingerAccelerationModel
from tracking_v2.sensor import GeometricSensor
from tracking_v2.evaluation import Runner, evaluate_nees, plot_error, plot_3d

from tracking.util import to_df

In [ ]:
target = SingleTurnTarget()
sensor = GeometricSensor()

motion = ConstantVelocityModel(.0001)
kf = LinearKalmanFilter(motion, [[1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0]])

r = Runner(target, sensor, kf)
r.run_many(1, 500)

In [ ]:
plot_3d(r)

In [ ]:
plot_error(r)

In [ ]:
target = SingleTurnTarget()
sensor = GeometricSensor()

motion = ConstantVelocityModel(20)
kf = LinearKalmanFilter(motion, [[1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0]])

r = Runner(target, sensor, kf)
r.run_many(1, 500)

In [ ]:
plot_3d(r)

In [ ]:
plot_error(r)

In [ ]:
r = Runner(target, sensor, kf)
r.run_many(100, 500)

In [ ]:
np.max(r.many_x_hat[:,:,:3,0] - r.truth[1:,:3], axis=2).max(axis=1).max()

# Accumulation of error

The buildup of Normalized Estimation Error Squared (NEES) in individual runs of a Kalman Filter, despite the mean across multiple runs being statistically consistent with the expected confidence intervals, can be due to several factors:

* Model-Data Mismatch: Even when you assume zero process noise, there might be unaccounted-for model inaccuracies or if the initial conditions are not perfectly set. Since the target moves with constant velocity, any small deviation from this assumption due to initial state errors or numerical precision issues might accumulate over time.
* Numerical Stability: Numerical issues in implementation, such as round-off errors or the propagation of small errors through the covariance matrix calculations, could lead to an increase in NEES over time. 
* Statistical Fluctuations: In individual runs, due to the stochastic nature of the noise, you might observe outliers or sequences where errors accumulate before eventually averaging out over many runs.

Here are some considerations and potential solutions or mitigations:

* Non-zero Process Noise: Even if the process noise is theoretically zero, introducing a small non-zero process noise can help prevent the covariance from collapsing to zero. This can be justified by acknowledging that no model or system is perfectly deterministic in practice. 
  * Consider setting a very small but non-zero $Q$ (process noise covariance). This would keep the state covariance from going to zero, maintaining a more stable NEES.
* Covariance Inflation: Some approaches involve artificially inflating the covariance matrix to maintain numerical stability and performance. 
  * Techniques like covariance inflation or adding a small positive definite term to the covariance matrix after each update can be effective.
* State Augmentation: If not already done, you might augment your state to include additional parameters (like acceleration, even if you believe they should be zero) with some small variance, allowing for slight model mismatches or biases.
* Reset or Reinitialization: Periodically resetting or reinitializing the covariance might be considered if the filter's confidence becomes too high, but this must be done cautiously to not discard valuable information.
* Adaptive Filtering: Implement adaptive methods where the process noise or measurement noise covariance can be adjusted based on the observed performance of the filter. This can help in scenarios where the model might not perfectly capture the dynamics.

Literature on Filter Degeneracy: 

* Look into papers or books discussing "filter degeneracy" or "covariance collapse" in Kalman filtering. These often provide methods like those above or discuss the theoretical implications of zero process noise, e.g.:
  * "Stochastic Models, Estimation and Control" by Peter S. Maybeck, Volume 2, particularly the sections on adaptive filtering.
  * "Adaptive Filtering: Algorithms and Practical Implementation" by Paulo S.R. Diniz for insights into adaptive approaches that might be applicable.

Remember, in practical scenarios, even if the target moves with constant velocity, small unmodeled effects or measurement errors can accumulate, making zero process noise an idealized assumption. Addressing these issues might require a balance between theoretical fidelity and practical robustness.

In [ ]:
class MyRunner(Runner):
    x_trace = []
    i = 0
    
    def after_initialize(self):
        self.kf.x_hat[3,0] = 30
        pass
    
    def after_update(self, m):
        self.x_trace.append((self.i, self.kf.x_hat[0, 0], self.kf.x_hat[3, 0], m.error[0, 0]))
        self.i += 1
        
        # inflate covariance
        self.kf.P_hat += np.eye(6) * .0001

def _drift(q):
    target = ConstantVelocityTarget()
    motion = ConstantVelocityModel(q)
    kf = LinearKalmanFilter(motion, [[1, 0, 0, 0, 0, 0],
                                     [0, 1, 0, 0, 0, 0],
                                     [0, 0, 1, 0, 0, 0]])

    sensor = GeometricSensor(seed=8)
    r = MyRunner(target, sensor, kf)
    r.run_many(1, 500)
    return r

r = _drift(0)

In [ ]:
def _plot_position_error(runner):
    tm  = np.arange(runner.n-100).reshape((runner.n-100, -1))
    err = np.hstack((tm, np.abs(runner.one_x_hat[100:,:3,0] - runner.truth[101:,:3])))
    err = to_df(err, columns=['time', 'x', 'y', 'z']).melt(['time'], ['x', 'y', 'z'], 'dim', 'error')
    return ex.line(err, x='time', y='error', facet_row='dim')

_plot_position_error(r)

In [ ]:
def _plot_velocity_error(runner, skip=100):
    tm  = np.arange(runner.n-skip).reshape((runner.n-skip, -1))
    err = np.hstack((tm, np.abs(runner.one_x_hat[skip:,3:6,0] - runner.truth[(skip+1):,3:6])))
    err = to_df(err, columns=['time', 'x', 'y', 'z']).melt(['time'], ['x', 'y', 'z'], 'dim', 'error')
    return ex.line(err, x='time', y='error', facet_row='dim')

_plot_velocity_error(r, 0)

In [ ]:
def _plot_position_nees(runner):
    nees = evaluate_nees(runner.one_x_hat[:, :3, :], runner.one_P_hat[:, :3, :3], runner.truth[1:, :3])
    err = np.asarray((np.arange(runner.n-100), nees.scores[100:])).T
    err = to_df(err, columns=['time', 'nees'])
    fig = ex.line(err, x='time', y='nees')
    
    ci = sp.stats.chi2.ppf([0.025, 0.975], nees.dim)
    fig.add_hline(y=ci[0], line_width=.5, line_dash="dash", line_color="red")
    fig.add_hline(y=ci[1], line_width=.5, line_dash="dash", line_color="red")

    return fig

_plot_position_nees(r)

In [ ]:
d = np.array(r.x_trace)
d[:, 1] -= r.truth[1:, 0]

d = to_df(d, columns=['time', 'x', 'v', 'me'])

In [ ]:
ex.scatter(d.melt(['time'], ['x', 'v', 'me'], 'dim', 'value'), x='time', y='value', color='dim')